# 3절. Selenium을 이용한 웹 데이터 수집
- 구글에서 'selenium python'검색
https://selenium-python.readthedocs.io/getting-started.html
- pip install selenium

- **chrome버전을 확인하고 맞는 chrome driver버전을 다운로드** 

In [1]:
from selenium import webdriver
driver= webdriver.Chrome("D:/bigdata(psa)/selenium/chromedriver.exe")    # /하나 혹은 \\ (역슬래시 두 개)
#chromedriver.exe가 쥬피터노트북파일 여기서  bigdata(psa) src  8_python안에 있으면  Chrome() 안에 아무것도 안써도됨 

In [2]:
driver.get('http://python.org')

In [5]:
elem=driver.find_element_by_name('q')   #find에  shift+tab 키   
elem.clear()   # 검색창안에 입력한 내용을 지우고 
elem.send_keys("pycon")    #그러면 브라우저 검색창에서 pycon이 나타남 

In [6]:
#pycon를 쳤으면 enter를 눌러라
from selenium.webdriver.common.keys import Keys
elem.send_keys(Keys.RETURN) # Enter키 

In [9]:
result_list=driver.find_elements_by_css_selector("form h3>a")   #결과 여러개 있어서 find_element가 아니고  find_elements  
for result in result_list:
    print("{%s-%s"%(result.text,result.get_attribute("href")))

{PSF PyCon Trademark Usage Policy-https://www.python.org/psf/trademarks/pycon
{Conferences and Workshops-https://www.python.org/community/workshops
{PyCon Italia 2016 (PyCon Sette)-https://www.python.org/events/python-events/378/
{2008-04-14 PSF Board Meeting Minutes-https://www.python.org/psf/records/board/minutes/2008-04-14
{2012-07-16 PSF Board Meeting Minutes-https://www.python.org/psf/records/board/minutes/2012-07-16
{2013-02-06 PSF Board Meeting Minutes-https://www.python.org/psf/records/board/minutes/2013-02-06
{PyCon Home at python.org-https://www.python.org/community/pycon
{PyCon Australia 2013-https://www.python.org/events/python-events/57/
{PyCon AU 2019-https://www.python.org/events/python-events/776/
{PyCon Australia 2014-https://www.python.org/events/python-events/10/
{PyCon Ireland 2012-https://www.python.org/events/python-events/76/
{PyCon Ireland 2016-https://www.python.org/events/python-events/429/
{PyCon AU 2018-https://www.python.org/events/python-events/696/
{PyCon

In [10]:
#브라우저 종료
driver.close()